In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("librispeech_asr",
                       split="train.clean.100",
                       streaming=True,
                       trust_remote_code=True)

In [ ]:
example = next(iter(dataset))

In [ ]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(example["audio"]["array"],
             rate=example["audio"]["sampling_rate"])

In [ ]:
from transformers import pipeline
asr = pipeline(task="automatic-speech-recognition", 
               model="distil-whisper/distil-small.en")

In [ ]:
asr.feature_extractor.sampling_rate

In [ ]:
example['audio']['sampling_rate']

In [ ]:
asr(example["audio"]["array"])

In [ ]:
example["text"]

In [ ]:
import os
import gradio as gr
demo = gr.Blocks()

In [ ]:
#import gradio as gr

#demo = gr.Blocks()

#with demo:
 #   gr.Markdown("# Hello, Gradio!")  
  #  name = gr.Textbox(label="Enter your name")  
   # output = gr.Textbox(label="Output")  

    #def greet(name):  
     #   return f"Hello, {name}!"

   # name.submit(fn=greet, inputs=name, outputs=output)  # Link the function to UI elements

#demo.launch()

In [ ]:
def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(filepath)
    return output["text"]
 

In [ ]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never")

In [ ]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never",
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe,
         file_transcribe],
        ["Transcribe Microphone",
         "Transcribe Audio File"],
    )

demo.launch(share=True, 
            server_port=int(os.environ['PORT1']))